In [1]:
sample_cdf = {
	"task_desc": "",
	"scene": {
		"scene_type": [],
		"simbot_init": [],
		"required_objects": [
		]
	},
	"task_goals": []
}

In [2]:
sample_cdf["scene"]["required_objects"]

[]

In [3]:
sample_desc = [(('office', 'in', '__room__'), ({'det': 'an'}, {})),
 (('window', 'in', '__room__'), ({'det': 'a', 'amod': 'large'}, {})),
 (('desk', 'in', '__room__'), ({'det': 'a', 'amod': 'wooden'}, {})),
 (('laptop', 'on', 'desk'), ({'det': 'a'}, {})),
 (('chair', 'in front of', 'desk'),
  ({'det': 'an', 'compound': 'office'}, {'det': 'the'})),
 (('table', 'in', '__room__'),
  ({'det': 'a', 'amod': 'small', 'compound': 'coffee'}, {}))]

In [4]:
furniture_count = {}

In [5]:
def Desc2Ai2ThorName(desc):
    object_name = desc[0][0]
    relation = desc[0][1]
    place_name = desc[0][2]
    
    attr = desc[1]
    
    object_name = object_name.capitalize()  # uppercase first letter
    place_name = place_name.capitalize()
    
    if 'compound' in attr[0]:
        comp = attr[0]['compound']
        comp = comp.capitalize()
        if object_name not in ["Chair"]:
            object_name = comp + object_name
        
    return object_name, relation, place_name

In [6]:
Desc2Ai2ThorName( (('laptop', 'on', 'desk'), ({'det': 'a'}, {})))

('Laptop', 'on', 'Desk')

In [7]:
for desc in sample_desc:
    print(desc)
    object_name, relation, place_name = Desc2Ai2ThorName(desc)
    
    # resolve room type
    if object_name in ["Office","Bedroom"]:
        if place_name == "__room__" or place_name == "room":
            sample_cdf["scene"]["scene_type"].append(1 if object_name == "Office" else 2)
    
    # resolve furniture
    elif place_name == "__room__" or place_name == "room":
        object_index = furniture_count.get(object_name, 0) + 1
        
        object_info = {
            "name": "{}_{}".format(object_name, str(object_index)),
        }
        
        sample_cdf["scene"]["required_objects"].append(object_info)
        furniture_count[object_name] = object_index
    
    # resolve object on furniture
    elif relation in ["on", "in front of"]:
        object_index = furniture_count.get(object_name, 0) + 1
        
        on_furniture = None
        for obj in sample_cdf["scene"]["required_objects"]:
            if obj["name"].split('_')[0] == place_name:
                on_furniture = obj["name"]
        
        if on_furniture is None:
            raise("No furniture found")
        
        relation2thor = relation if relation == "on" else "face"
        
        object_info = {
            "name": "{}_{}".format(object_name, str(object_index)),
            "location": [{
            "{}".format(on_furniture): "{}".format(relation2thor)
            }]
        }
        
            
        sample_cdf["scene"]["required_objects"].append(object_info)
        furniture_count[object_name] = object_index

(('office', 'in', '__room__'), ({'det': 'an'}, {}))
(('window', 'in', '__room__'), ({'det': 'a', 'amod': 'large'}, {}))
(('desk', 'in', '__room__'), ({'det': 'a', 'amod': 'wooden'}, {}))
(('laptop', 'on', 'desk'), ({'det': 'a'}, {}))
(('chair', 'in front of', 'desk'), ({'det': 'an', 'compound': 'office'}, {'det': 'the'}))
(('table', 'in', '__room__'), ({'det': 'a', 'amod': 'small', 'compound': 'coffee'}, {}))


In [8]:
sample_cdf

{'task_desc': '',
 'scene': {'scene_type': [1],
  'simbot_init': [],
  'required_objects': [{'name': 'Window_1'},
   {'name': 'Desk_1'},
   {'name': 'Laptop_1', 'location': [{'Desk_1': 'on'}]},
   {'name': 'Chair_1', 'location': [{'Desk_1': 'face'}]},
   {'name': 'CoffeeTable_1'}]},
 'task_goals': []}

In [9]:
import json